## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-16-13-00-00 +0000,missionlocal,Check me out,https://missionlocal.org/2025/08/sf-photos-che...
1,2025-08-16-12-55-33 +0000,nyt,Environmental Groups Face ‘Generational’ Setba...,https://www.nytimes.com/2025/08/16/climate/env...
2,2025-08-16-12-55-01 +0000,startribune,Ukraine's Zelenskyy to meet Trump on Monday af...,https://www.startribune.com/trump-speaks-to-uk...
3,2025-08-16-12-45-00 +0000,nypost,Adam Schiff paying just 3% mortgage on homes a...,https://nypost.com/2025/08/16/us-news/adam-sch...
4,2025-08-16-12-33-26 +0000,nyt,Live Updates: Trump Backs Putin’s Route for Ta...,https://www.nytimes.com/live/2025/08/16/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2357/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,71
32,putin,44
10,ukraine,20
15,summit,17
577,meeting,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
14,2025-08-16-11-41-01 +0000,nyt,"After Trump-Putin Alaska Summit, Europeans Wor...",https://www.nytimes.com/2025/08/16/world/europ...,230
219,2025-08-15-15-56-48 +0000,nyt,"How Trump’s Meeting With Putin Could Unfold, a...",https://www.nytimes.com/2025/08/15/us/politics...,229
242,2025-08-15-13-25-54 +0000,nypost,Putin’s foreign minister trolls Ukraine by wea...,https://nypost.com/2025/08/15/world-news/putin...,209
54,2025-08-16-08-01-18 +0000,nyt,Trump and Putin had an isolated conversation i...,https://www.nytimes.com/live/2025/08/15/world/...,192
118,2025-08-16-00-14-50 +0000,nypost,"Putin tries to woo Trump at Alaska meeting, cl...",https://nypost.com/2025/08/15/us-news/putin-tr...,185


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
14,230,2025-08-16-11-41-01 +0000,nyt,"After Trump-Putin Alaska Summit, Europeans Wor...",https://www.nytimes.com/2025/08/16/world/europ...
159,64,2025-08-15-20-58-21 +0000,nypost,Trump reacts to Hillary Clinton saying she’d n...,https://nypost.com/2025/08/15/us-news/trump-re...
79,45,2025-08-16-03-57-56 +0000,nyt,Trump Administration Backs Off New Attempt to ...,https://www.nytimes.com/2025/08/15/us/washingt...
243,36,2025-08-15-13-24-43 +0000,cbc,Canada Post and union meeting pushed to next w...,https://www.cbc.ca/news/politics/canada-post-u...
183,31,2025-08-15-19-20-08 +0000,nypost,High-ranking Hamas terrorist killed by IDF ami...,https://nypost.com/2025/08/15/world-news/hamas...
68,29,2025-08-16-05-12-25 +0000,nypost,California babysitter weeps as she’s sentenced...,https://nypost.com/2025/08/16/us-news/californ...
2,28,2025-08-16-12-55-01 +0000,startribune,Ukraine's Zelenskyy to meet Trump on Monday af...,https://www.startribune.com/trump-speaks-to-uk...
66,28,2025-08-16-05-45-17 +0000,nypost,"Air Canada cabin staff go on strike, grounding...",https://nypost.com/2025/08/16/world-news/air-c...
187,28,2025-08-15-18-27-02 +0000,nypost,Feds accuse New Orleans Mayor LaToya Cantrell ...,https://nypost.com/2025/08/15/us-news/mayor-la...
59,23,2025-08-16-06-45-52 +0000,nypost,"Senior, 76, died while trying to meet Meta AI ...",https://nypost.com/2025/08/16/us-news/nj-senio...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
